In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
df1 = pd.read_csv('.././3G_with_anomaly_classes.csv')
df2 = pd.read_csv('.././DataFrame_3G_After_Cleaning.csv')
df3 = pd.read_csv('.././DataFrame_3G_After_Cleaning.csv')

Isolation Forest

In [22]:
import plotly.express as px
from sklearn.ensemble import IsolationForest
import plotly.graph_objects as go

In [23]:
x = df1['PS DROP Rate 2'].apply(lambda x: [x]).to_list()
iso_forest = IsolationForest(n_estimators=100,
                             max_samples="auto",
                             contamination=0.05,
                             )
iso_forest.fit(x)
y_pred1 = iso_forest.predict(x)
y_pred1 = [1 if x == -1 else 0 for x in y_pred1]
y_pred1[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [24]:
df1["PSDR_anomaly"] = y_pred1
#df.head()

In [25]:
iso_anomaly_df = pd.DataFrame(df1)
iso_anomaly_df = iso_anomaly_df.loc[iso_anomaly_df['PSDR_anomaly'] == 1]
#iso_anomaly_df.head()


In [26]:
fig = px.line(df1,x='index', y='PS DROP Rate 2', title='Unsupervised IForest anomaly detection in PS DROP Rate ')
fig.add_trace(go.Scatter(x=iso_anomaly_df["index"].to_list(), y=iso_anomaly_df["PS DROP Rate 2"].to_list(),
                            mode='markers', name='anomalies'))
fig.update_xaxes(rangeslider_visible=True)
fig

In [27]:
nbr_val_inf_threshold_PSDR = len(df1[df1['PS DROP Rate 2'] > 3.00])
print("Nombres des valeurs inférieur à 2.00% sont :", nbr_val_inf_threshold_PSDR)
ifor_nbr_all_anomaly_PSDR = len(df1[df1['PSDR_anomaly'] == 1])
ifor_nbr_true_anom_PSDR = len(df1[(df1['PSDR_anomaly'] == 1) & (df1['PS DROP Rate 2'] > 3.00)])
print("Nombres des vrais anomalie détecter sont :",ifor_nbr_true_anom_PSDR," parmi ",ifor_nbr_all_anomaly_PSDR," prédis")
%store nbr_val_inf_threshold_PSDR
%store ifor_nbr_all_anomaly_PSDR
%store ifor_nbr_true_anom_PSDR

Nombres des valeurs inférieur à 2.00% sont : 14
Nombres des vrais anomalie détecter sont : 9  parmi  12  prédis
Stored 'nbr_val_inf_threshold_PSDR' (int)
Stored 'ifor_nbr_all_anomaly_PSDR' (int)
Stored 'ifor_nbr_true_anom_PSDR' (int)


In [28]:
df1.to_csv('3G_with_anomaly_classes.csv')

Local Outlier Factor

In [29]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(n_neighbors=10, contamination=0.05)
y_pred2 = lof.fit_predict(x)
y_pred2 = [1 if x == -1 else 0 for x in y_pred2]
y_pred2[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [30]:
df2["PSDR_anomaly"] = y_pred2
#df.head()

In [31]:
lof_anomaly_df = pd.DataFrame(df2)
lof_anomaly_df = lof_anomaly_df.loc[lof_anomaly_df['PSDR_anomaly'] == 1]
#lof_anomaly_df.head()

In [32]:
fig = px.line(df2, x='index', y='PS DROP Rate 2', title='Unsupervised anomaly LOF detection in PS DROP Rate 2')
fig.add_trace(go.Scatter(x=lof_anomaly_df["index"].to_list(), y=lof_anomaly_df["PS DROP Rate 2"].to_list(),
                        mode='markers', name='anomalies'))
fig.update_xaxes(rangeslider_visible=True)
fig

In [33]:
nbr_val_inf_threshold = len(df2[df2['PS DROP Rate 2'] > 3.00])
print("Nombres des valeurs inférieur à 2.00% sont :", nbr_val_inf_threshold)
lof_nbr_all_anomaly_PSDR = len(df2[df2['PSDR_anomaly'] == 1])
lof_nbr_true_anom_PSDR = len(df2[(df2['PSDR_anomaly'] == 1) & (df2['PS DROP Rate 2'] > 3.00)])
print("Nombres des vrais anomalie détecter sont :",lof_nbr_true_anom_PSDR," parmi ",lof_nbr_all_anomaly_PSDR," prédis")
%store lof_nbr_all_anomaly_PSDR
%store lof_nbr_true_anom_PSDR

Nombres des valeurs inférieur à 2.00% sont : 14
Nombres des vrais anomalie détecter sont : 4  parmi  12  prédis
Stored 'lof_nbr_all_anomaly_PSDR' (int)
Stored 'lof_nbr_true_anom_PSDR' (int)


OneClassSVM

In [34]:
from sklearn.svm import OneClassSVM
ocsvm = OneClassSVM(kernel='rbf',
                     gamma="auto",
                     nu=0.05)
y_pred3 = ocsvm.fit_predict(x)
y_pred3 = [1 if x == -1 else 0 for x in y_pred3]
y_pred3[:10]

[0, 0, 0, 1, 1, 0, 0, 0, 1, 0]

In [35]:
df3["PSDR_anomaly"] = y_pred3
df3.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,PSDR_anomaly
0,0,16-12-21,0,74.39,66.74,7.65,156.83,99.69,0.77,95.94,1.72,100.0,99.86,93.94,100.00,755.79,0
1,1,17-12-21,0,65.41,58.40,7.01,107.82,99.26,0.97,95.40,2.37,100.0,99.67,97.98,100.00,755.79,0
2,2,18-12-21,0,65.67,58.93,6.74,94.38,99.26,1.09,94.97,3.03,100.0,99.57,96.43,0.00,755.79,0
3,3,19-12-21,0,57.79,50.91,6.87,101.48,99.08,1.23,95.43,2.60,100.0,99.72,93.14,100.00,755.79,1
4,4,20-12-21,0,79.74,70.04,9.69,163.22,99.65,0.76,95.16,1.61,100.0,99.86,94.23,66.67,755.79,1


In [36]:
ocsvm_anomaly_df = pd.DataFrame(df3)
ocsvm_anomaly_df = ocsvm_anomaly_df.loc[ocsvm_anomaly_df['PSDR_anomaly'] == 1]
ocsvm_anomaly_df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,PSDR_anomaly
3,3,19-12-21,0,57.79,50.91,6.87,101.48,99.08,1.23,95.43,2.60,100.00,99.72,93.14,100.00,755.79,1
4,4,20-12-21,0,79.74,70.04,9.69,163.22,99.65,0.76,95.16,1.61,100.00,99.86,94.23,66.67,755.79,1
8,8,24-12-21,0,66.24,58.16,8.07,147.24,99.36,3.93,95.59,2.61,93.75,99.63,94.68,100.00,696.83,1
10,10,26-12-21,0,61.63,54.18,7.46,92.98,99.14,3.44,95.36,3.75,97.34,99.62,94.97,50.00,639.02,1
24,24,01-09-22,0,72.46,61.93,10.53,89.79,98.41,3.48,93.11,4.73,96.95,98.69,92.60,100.00,490.41,1


In [37]:
fig = px.line(df3, x='index', y='PS DROP Rate 2', title='Unsupervise OC-SVM anomaly detection in PS DROP Rate 2')
fig.add_trace(go.Scatter(x=ocsvm_anomaly_df["index"].to_list(),
                        y=ocsvm_anomaly_df["PS DROP Rate 2"].to_list(),
                        mode='markers',
                        name='anomaly'))
fig.update_xaxes(rangeslider_visible=True)
fig

In [38]:
nbr_val_inf_threshold = len(df3[df3['PS DROP Rate 2'] > 3.00])
print("Nombres des valeurs inférieur à 3.00% sont :", nbr_val_inf_threshold)
ocsvm_nbr_all_anomaly_PSDR = len(df3[df3['PSDR_anomaly'] == 1])
ocsvm_nbr_true_anom_PSDR = len(df3[(df3['PSDR_anomaly'] == 1) & (df3['PS DROP Rate 2'] > 3.00)])
print("Nombres des vrais anomalie détecter sont :",ocsvm_nbr_true_anom_PSDR," parmi ",ocsvm_nbr_all_anomaly_PSDR," prédis")
%store ocsvm_nbr_all_anomaly_PSDR
%store ocsvm_nbr_true_anom_PSDR

Nombres des valeurs inférieur à 2.00% sont : 14
Nombres des vrais anomalie détecter sont : 3  parmi  13  prédis
Stored 'ocsvm_nbr_all_anomaly_PSDR' (int)
Stored 'ocsvm_nbr_true_anom_PSDR' (int)
